In [1]:
import sys

sys.path.append("..")
from trading.system import TradingEnv

/home/aaruran/miniconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [1]:
import math
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

/home/aaruran/miniconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def validate_numeric_list(x):
    assert not any(map(math.isnan, x)), "Found nan in x."
    return True

def basic_loop(t, *args, **kwargs):
    env = TradingEnv(ticker=t, *args, **kwargs)
    state = env.reset()
    assert env.prices[:env.WINDOW_SIZE].std() > 0, "Standard Deviation too low"
    validate_numeric_list(state)
    trading_data = env.data[env.df_initial_index - env.WINDOW_SIZE : env.df_final_index]
    assert_frame_equal(trading_data, trading_data.fillna(np.inf))
    done = False
    while not done:
        next_state, r, done, _ = env.step(0)
    h = env.close()
    assert_frame_equal(h, h.fillna(np.inf))


In [3]:
basic_loop('AAPL', mode="test")

In [4]:
clean_stocks = []
failed_to_load_list = []
with open('raw_sp500.tsv', 'r') as src:
    for i, line in enumerate(src.readlines()):
        if i == 0:
            continue
        ticker = line.split()[0]
        ticker = ticker.replace('.', '-')
        try:
            basic_loop(ticker)
            basic_loop(ticker, mode='test')
            clean_stocks.append(ticker)
        except:
            print(f'failed on {ticker}')
            failed_to_load_list.append(ticker)
            

failed on ALGN
failed on GOOGL
failed on AMCR
failed on ANET
failed on BDX
failed on CARR
failed on CBOE
failed on CBRE
failed on CFG
failed on CME
failed on CMA
failed on CTVA
failed on CVS
failed on DLR
failed on DG
failed on DOW
failed on EOG
failed on EQIX
failed on EVRG
failed on EXPD
failed on FE
failed on FLT
failed on FLIR
failed on FTV
failed on FBHS
failed on FOXA
failed on FOX
failed on FCX
failed on GPS
failed on GD
failed on HAS
failed on HPE
failed on HLT
failed on HD
failed on HON
failed on HWM
failed on HII
failed on INFO
failed on ICE
failed on IBM
failed on IFF
failed on ISRG
failed on IVZ
failed on IPGP
failed on JKHY
failed on JBHT
failed on KSU
failed on K
failed on KEYS
failed on KMI
failed on KHC
failed on KR
failed on LHX
failed on LW
failed on LKQ
failed on MMC
failed on MET
failed on MAA
failed on MHK
failed on TAP
failed on MDLZ
failed on MS
failed on NDAQ
failed on NOV
failed on NFLX
failed on NEM
failed on NEE
failed on NLSN
failed on NBL
failed on NOC
fail

In [6]:
assert len(clean_stocks) == 386, "clean stocks missing. check golden files"

In [7]:
len(clean_stocks)

386

In [12]:
failed_to_load_list.sort()
clean_stocks.sort()

In [13]:
with open('blacklist.txt', 'w') as target:
    target.write('\n'.join(failed_to_load_list))

In [14]:
with open('filtered_tickers.txt', 'w') as target:
    target.write('\n'.join([s for s in clean_stocks]))